# MCQ (Multiple Choice Question) 평가 튜토리얼

이 튜토리얼에서는 LLM을 사용하여 객관식 문제를 평가하는 방법을 배워보겠습니다.

## 주요 기능
- 객관식 문제의 테스트 케이스 생성
- LLM을 사용한 문제 해결
- 다국어 지원 (한국어/영어)
- 비동기 실행 지원

## 1. 필요한 라이브러리 임포트

In [1]:
from langmetrics.llmfactory import LLMFactory

## 2. MCQ 테스트 케이스 정의

## MCQTestCase 클래스 정의
MCQTestCase 클래스는 다음과 같은 필드로 구성되어 있습니다. 필드의 의미는 다음과 같습니다:
- input: LLM에 입력할 질문
- choices: 선택 가능한 답변들의 리스트
- expected_output: 정답 (인덱스 또는 문자열)
- output: LLM이 실제로 출력한 답변 (선택사항)
- reasoning: LLM의 답변 도출 과정 (선택사항)

## 3. 테스트 케이스 생성 예시
간단한 퀴즈 문제를 만들어 MCQTestCase를 사용해보겠습니다.

In [2]:
from langmetrics.llmtestcase import MCQTestCase
# 테스트 케이스 예시 생성
simple_testcase = MCQTestCase(
    input="프랑스의 수도는 어디입니까?",
    choices=["런던", "파리", "베를린", "마드리드"],
    expected_output="B"
)

medical_testcase = MCQTestCase(
    input="임신 22주차인 23세 임산부가 배뇨 시 통증을 호소합니다. 1일 전부터 시작되었으며 수분 섭취 증가와 크랜베리 추출물 복용에도 불구하고 악화되고 있습니다. 체온 36.5°C, 혈압 122/77mmHg, 맥박 80회/분, 호흡수 19회/분, 산소포화도 98%입니다. 신체검사상 척추각 압통은 없으며 임신한 자궁이 관찰됩니다. 가장 적절한 치료는 무엇입니까?",
    choices=["Ampicillin", "Ceftriaxone", "Ciprofloxacin", "Nitrofurantoin"],
    expected_output="A",
    output="A"
)

# 테스트 케이스 출력
print("일반상식 문제:")
print(f"질문: {simple_testcase.input}")
print(f"선택지: {simple_testcase.choices}")
print(f"정답 인덱스: {simple_testcase.expected_output}")
print("\n의료 문제:")
print(f"질문: {medical_testcase.input}")
print(f"선택지: {medical_testcase.choices}")
print(f"정답: {medical_testcase.expected_output}")
print(f"추론: {medical_testcase.output}")

일반상식 문제:
질문: 프랑스의 수도는 어디입니까?
선택지: ['런던', '파리', '베를린', '마드리드']
정답 인덱스: B

의료 문제:
질문: 임신 22주차인 23세 임산부가 배뇨 시 통증을 호소합니다. 1일 전부터 시작되었으며 수분 섭취 증가와 크랜베리 추출물 복용에도 불구하고 악화되고 있습니다. 체온 36.5°C, 혈압 122/77mmHg, 맥박 80회/분, 호흡수 19회/분, 산소포화도 98%입니다. 신체검사상 척추각 압통은 없으며 임신한 자궁이 관찰됩니다. 가장 적절한 치료는 무엇입니까?
선택지: ['Ampicillin', 'Ceftriaxone', 'Ciprofloxacin', 'Nitrofurantoin']
정답: A
추론: A


## 4. LLM을 이용하여 MCQ 평가

LLM을 설정하고 MCQ 평가를 실행합니다.

In [3]:
# LLM 모델 생성
deepseek_llm = LLMFactory.create_llm('deepseek-v3')

평가를 위하여 MCQMetric class를 호출하도록 하겠습니다. MCQMetric은 LLM(Large Language Model)의 객관식 문제 답변을 평가하기 위한 메트릭 클래스입니다.

MCQMetric을 사용하기 위해서는 먼저 인스턴스를 생성해야 합니다. 기본적인 설정 방법은 다음과 같습니다:

### 주요 매개변수 :

`template_language`: 템플릿 언어 선택 ('ko' 또는 'en')

`generate_template_type`: 답변 생성 방식 ('reasoning': 풀이 과정 포함, 'only_answer': 답만 생성)

`verbose_mode`: 상세 로그 출력 여부 (기본값: False)

In [4]:
from langmetrics.metrics import MCQMetric
metric = MCQMetric(
    answer_model=deepseek_llm,
    template_language='en',  # 'ko' 또는 'en'
    generate_template_type='reasoning'  # 'reasoning' 또는 'only_answer'
)

In [5]:
result = metric.measure(simple_testcase)

case validate 검증 완료
```json
{
    "reasoning": "프랑스의 수도는 파리입니다. 런던은 영국의 수도, 베를린은 독일의 수도, 마드리드는 스페인의 수도입니다. 따라서 정답은 파리입니다. So the answer is B.",
    "answer": "B"
}
```


input에 들어가 simple_testcase는 정답값과 함께 출력이 됨을 확인해주세요.

In [6]:
simple_testcase

MCQTestCase(input='프랑스의 수도는 어디입니까?', choices=['런던', '파리', '베를린', '마드리드'], expected_output='B', output='B', reasoning='프랑스의 수도는 파리입니다. 런던은 영국의 수도, 베를린은 독일의 수도, 마드리드는 스페인의 수도입니다. 따라서 정답은 파리입니다. So the answer is B.')

In [7]:
print(result)

문제: 프랑스의 수도는 어디입니까?
선택지: ['런던', '파리', '베를린', '마드리드']
정답: B
결과: 정답
추론: 프랑스의 수도는 파리입니다. 런던은 영국의 수도, 베를린은 독일의 수도, 마드리드는 스페인의 수도입니다. 따라서 정답은 파리입니다. So the answer is B.
토큰 사용량: {'completion_tokens': 70, 'prompt_tokens': 151, 'total_tokens': 221}


MCQResult는 to_dict()와 from_dict() method를 지원합니다.

In [8]:
result_dict = result.to_dict()

In [9]:
result.from_dict(result_dict)

MCQResult(question='프랑스의 수도는 어디입니까?', predicted='B', language='en', score=1, ground_truth='B', choice=['런던', '파리', '베를린', '마드리드'], reasoning="The question asks for the capital of France. Let's analyze the options step by step: A. 런던 (London) is the capital of the United Kingdom, not France. B. 파리 (Paris) is the correct capital of France. C. 베를린 (Berlin) is the capital of Germany, not France. D. 마드리드 (Madrid) is the capital of Spain, not France. So the answer is B.", token_usage={'completion_tokens': 112, 'prompt_tokens': 150, 'total_tokens': 262})

## 3. MCQ 템플릿 설정


langmetrics의 Metric은 custom template을 이용할 수 있고, 이 template은 langchain의 Template을 이용하여 사용할 수 있습니다.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langmetrics.metrics import MCQTemplate

메시지를 설정하되, 반드시 JSON을 output을 출력으로 뱉는 template을 설정해줍니다.

In [11]:
evaluation_prompt = ChatPromptTemplate.from_messages([
    AIMessagePromptTemplate.from_template('당신은 의료 전문가입니다.'),
    HumanMessagePromptTemplate.from_template("""다음의 객관식 문제를 풀어주세요.
추론 과정을 설명하고 정답은 알파벳(A, B, C, D 등)으로만 답해주세요.

**
중요 : 반드시 JSON 형식으로만 답변해주세요. 'answer' 키에는 정답을 작성해주세요.
JSON 예시:
{{
"answer": "<정답>"
}}
**

문제:
{question}

보기:
{choices}

JSON:"""),
])


MCQTemplate 클래스는 정의된 프롬프트를 사용하여 객관식 문제 템플릿을 생성합니다.

In [12]:
answer_template = MCQTemplate(prompt_for_answer=evaluation_prompt)

In [13]:
metric = MCQMetric(
    deepseek_llm, 
    verbose_mode=True, 
    template_language='en',
    template=answer_template
)

In [15]:
print(metric.measure(simple_testcase))

문제: 프랑스의 수도는 어디입니까?
선택지: ['런던', '파리', '베를린', '마드리드']
정답: B
결과: 정답
추론: The question asks for the capital of France. Let's analyze the options step by step: A. 런던 (London) is the capital of the United Kingdom, not France. B. 파리 (Paris) is the correct capital of France. C. 베를린 (Berlin) is the capital of Germany, not France. D. 마드리드 (Madrid) is the capital of Spain, not France. So the answer is B.
토큰 사용량: None
